In [1]:
from Solid.ParticleSwarm import ParticleSwarm
from sklearn.metrics import mean_squared_error
from tengp.individual import IndividualBuilder, NPIndividual
from tengp import Parameters, FunctionSet
import numpy as np

np.warnings.filterwarnings('ignore')

In [68]:
def fitness(individual, x, y):
    output = individual.transform(x)
    try:
        return mean_squared_error(output, y)
    except ValueError:
        return 10e10

class Algorithm(ParticleSwarm):
    """
    Optimize the CCGP individual
    """
    def custom_init(self,
                    params,
                    bounds,
                    x_train,
                    y_train,
                    fitness_fn,
                    predictors, trainers):
        self.cache = {}
        self.params = params
        self.bounds = bounds
        self.x_train_full = x_train
        self.y_train_full = y_train
        self.fitness_fn = fitness_fn
        self.predictors = predictors
        self.trainers = trainers
        self.predictors.evaluate_fitness(self.trainers)
        self.x_train_pred, self.y_train_pred = self.predictors.best_predictors_data()
        
    
    def _objective(self, member):
        """
        Calculate the fitness (the less, the better)
        """
        member = np.clip(member, self.lower_bound, self.upper_bound)
        
        key = hash(member.tostring())
        
        if key in self.cache:
            return self.cache[key]
     
        individual = NPIndividual(member.tolist(), self.bounds, self.params)
        
        fitness = self.fitness_fn(individual, self.x_train_pred, self.y_train_pred)
        
        self.cache[key] = fitness
        
        return fitness

    def _generation_callback(self):
        # add new trainer
        if self.cur_steps % 50 == 0:
            #import pdb; pdb.set_trace()
            genes = np.clip(self.global_best[0], self.bounds[0], self.bounds[1]).tolist()
            individual = NPIndividual(genes, self.bounds, self.params)
            individual.fitness = self.fitness_fn(individual, self.x_train_full, self.y_train_full)
        
        # update random population
        if self.cur_steps % 50 == 0:
            ts.update_random_population()
            
        # evolution step of predictors
        if self.cur_steps % 50 == 0:
            self.predictors.predictors_evolution_step(self.trainers)
            self.x_train_pred, self.y_train_pred = self.predictors.best_predictors_data()
            
            
        #print(self.cur_steps)

In [69]:
from trainers import TrainersSet

ts = TrainersSet(ib, 8, fitness, x_train, y_train)

In [70]:
import ga

predictors = ga.GaPredictors(x_train, y_train, 4, 24)

In [71]:
# define CCGP system
from experiment_settings import nguyen7_funset
import symreg
import random

params = Parameters(2, 1, 1, 50, nguyen7_funset, real_valued=True, max_back=20)

ib = IndividualBuilder(params)
initial_individual = ib.create()
bounds = initial_individual.bounds
individual_size = len(initial_individual.genes)

In [72]:
x_train, y_train, _, _ = symreg.get_benchmark_nguyen7(random, None)
x_train = np.c_[np.ones(len(x_train)), x_train]

In [73]:
algo = Algorithm(swarm_size=50,
                member_size=individual_size,
                lower_bound=bounds[0],
                upper_bound=bounds[1],
                c1=1,
                c2=1,
                c3=1,
                max_steps=2000)

algo.custom_init(params, bounds, x_train, y_train, fitness, predictors, ts)

In [74]:
%%time

champion_x, champion_f = algo.run(verbose=False)

TERMINATING - REACHED MAXIMUM STEPS
CPU times: user 5min 35s, sys: 8min 8s, total: 13min 44s
Wall time: 3min 44s


In [75]:
champion_i = NPIndividual(np.clip(champion_x, bounds[0], bounds[1]).tolist(), bounds, params)
exact_fitness = fitness(champion_i, x_train, y_train)
print('predicted fitness', champion_f)
print('exact fitness', exact_fitness)

predicted fitness 0.00033810095699820223
exact fitness 1.6720174010314826
